In [1]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

# 1. Load the dataset
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

# 2. Check the shapes (Crucial Step!)
print(f"Features shape: {X.shape}")  # Should be (442, 10) -> 442 samples, 10 features
print(f"Target shape:   {y.shape}")  # Should be (442,) -> A flat vector

# 3. Split into Train and Test
# We train on 80% of the data, test on 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Features shape: (442, 10)
Target shape:   (442,)


In [2]:

def linreg(features_matrix,y, X_test, y_test) :
    y_vector = y.reshape(-1,1)
    # we get number of features we have in total
    nbr_features = features_matrix.shape[1]
    
    weights_vector = np.zeros(nbr_features).reshape(-1,1)
    intercept_scalar = 0
    
    #print("Shape of X:", features_matrix.shape)
    #print("Shape of y:", y_vector.shape)
    #print("Shape of w:", weights_vector.shape) 
    
    # rows,cols . cols,1 = rows,
    y_pred_vector = np.dot(features_matrix, weights_vector) + intercept_scalar
    
    #print("Shape of y_pred:", y_pred_vector.shape)
    
    error_vector = y_pred_vector - y_vector
    #print("error shape:", error_vector.shape)
    m_number_of_examples = features_matrix.shape[0]
    mse_vector = np.sum(error_vector**2)/m_number_of_examples

    dj_dy_pred = 1/m_number_of_examples * error_vector # y_pred_vector - y
    dy_pred_dw = features_matrix #X
    dy_pred_db = 1
    gd_iters = 0
    epsilon = 0.001
    alpha = 0.3

    #previous_params[0] is weights_vector   while  previous_params[1] is intercept_scalar
    previous_params = None

    
    while (True) : 
        # cols,rows  .  rows,1 = cols,1 
        dj_dw = np.dot(dy_pred_dw.T, dj_dy_pred) # chain rule
        #print("######")
        #print("weights_vector1:",weights_vector.shape)
        weights_vector = weights_vector - (alpha * dj_dw)
        #print("dj_dw",dj_dw.shape)
        #print("weights_vector2:",weights_vector.shape)
        #print("######")

        # summing the error vector into a scaler explicitly 
        # unlike in dj_dw it was  not needed cause dot product does it implicitly
        dj_db = np.sum(dj_dy_pred) * dy_pred_db # chain rule
        #print("######")
        #print("dj_db", dj_db.shape)
        intercept_scalar = intercept_scalar - (alpha * dj_db)
        #print("intercept_scalar", intercept_scalar.shape)
        #print("######")
        
        y_pred_vector = np.dot(features_matrix, weights_vector) + intercept_scalar
        error_vector = y_pred_vector - y_vector
        dj_dy_pred = 1/m_number_of_examples * error_vector
        previous_mse_vector = mse_vector 
        mse_vector = np.sum(error_vector**2)/m_number_of_examples
        
    
        # delta loss 
        cost_diff = previous_mse_vector - mse_vector 
        #print("cost_diff:", cost_diff)
        
        if cost_diff < 0:  # current_cost > prev_cost
            # revert back to previous params
            print("current weights_vector:",weights_vector.T)
            print("current intercept_scalar:",intercept_scalar)
            weights_vector = previous_params[0]
            intercept_scalar = previous_params[1]
            print("previous weights_vector:",weights_vector.T)
            print("previous intercept_scalar:",intercept_scalar)
            print(f"WARNING: cost increased from [previous_cost: {previous_mse_vector}] to [current_cost: {mse_vector}] at iteration [{gd_iters}]")
            # we should ecrease learning rate alpha
            break
        if ( cost_diff < epsilon ): 
            print(f"converged at iteration {gd_iters}")
            print("final training mse_vector:", mse_vector)
            print("final training cost_diff:", cost_diff)
            break 

        previous_params = [weights_vector, intercept_scalar]
        if gd_iters >= 10000 : 
            print("Gradient Descent reach max iterations [10000]")
            print("final training mse_vector:", mse_vector)
            print("final training cost_diff:", cost_diff)
            break
        gd_iters+=1

    y_test = y_test.reshape(-1,1)
    
    final_y_pred = (X_test @ weights_vector) + intercept_scalar 
    final_mse =  1/X_test.shape[0] * np.sum((final_y_pred - y_test)**2) 
    print("final test mse",final_mse)
    print("r^2 score", 1 - ( np.sum((y_test - final_y_pred)**2) / np.sum((y_test - y_test.mean())**2) ))

linreg(X_train, y_train,X_test,y_test)


converged at iteration 6509
final training mse_vector: 2898.1708163973576
final training cost_diff: 0.000999435941594129
final test mse 2879.872544907411
r^2 score 0.4564382672289956


In [3]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_test, y_test)

0.4526027629719195

# todos:
- I need to use OOP
- I need to try closed form solution for minima 
- train_test_split without using sklearn
- I need to add SGD and mini batch GD 
- I need to store mse in a list so i can plot learning curve 
- I need to implement poly
- I need to implement Regularization (Ridge/Lasso)
- GD with optimal step ?